# SwissPeaks Data

In this code, Swiss Peaks website is scraped. The raw data is then saved to a folder with race and year. No cleaning is done.


In [2]:
import requests
import json
import html
import pandas as pd
import time
import re
import unicodedata
import datetime
import random
import numpy as np
from datetime import timedelta

year = 2024  # Update as needed

# Start the timer
start_time = time.time()

year = '2024'

delay_seconds = 15

## Scraping Swiss Peaks 2024data from the website

In [4]:
# URL of the JSON file
url = f'https://www.ultratiming.live/_next/data/XCGRnFC4BCmNQ4CpCn1B0/en/evenement/spt-{year}/epreuve/360/resultats.json?eventSlug=spt-2024&trialSlug=360'
print(url)

# Send a GET request to fetch the JSON data
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()

    # Optionally, save the data to a JSON file
    with open(f'Data/JSON/SwissPeaks360_{year}.json', 'w') as f:
        json.dump(data, f, indent=4)

    print(f"Data saved to 'SwissPeaks360_{year}.json'")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

time.sleep(delay_seconds)

https://www.ultratiming.live/_next/data/XCGRnFC4BCmNQ4CpCn1B0/en/evenement/spt-2024/epreuve/360/resultats.json?eventSlug=spt-2024&trialSlug=360
Data saved to 'SwissPeaks_2024.json'


In [23]:
with open(f'Data/JSON/SwissPeaks360_{year}.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

    # Extract the participants list
    participants = json_data['pageProps']['resultsResult']['hydra:member']

    # Collect all extracted info in a list of dictionaries
    extracted_list = []

    for p in participants:
        user = p['user']
        
        extracted_info = {
            'firstName': user.get('firstName'),
            'lastName': user.get('lastName'),
            'slug': user.get('slug'),
            'nationality': user.get('nationality'),
            'gender': user.get('gender'),
            'birthDate': user.get('birthDate'),
            'distanceCarried': user.get('distanceCarried'),
            'participation_gender': p.get('gender'),
            'raceNumber': p.get('raceNumber'),
            'category': p.get('category'),
            'club': p.get('club')
        }

        extracted_list.append(extracted_info)

    # Convert to DataFrame
    df = pd.DataFrame(extracted_list)

# Show the first few rows
print(df.head())


  firstName     lastName                  slug nationality gender  \
0       Kim     COLLISON          collison-kim         GBR      M   
1  Matthieu  BOURGUIGNON  bourguignon-matthieu         FRA      M   
2     Jorge        GALVE           galve-jorge         ESP      M   
3    Adrien      BARBIER        barbier-adrien         FRA      M   
4     Iñigo      ORRADRE         orradre-inigo         ESP      M   

                   birthDate distanceCarried participation_gender raceNumber  \
0  1980-05-07T14:06:37+02:00             360                    M          2   
1  1979-03-26T14:07:05+01:00             360                    M         79   
2  1985-03-13T14:06:38+01:00             360                    M          5   
3  1989-02-01T09:51:16+01:00             360                    M         86   
4  1978-08-09T14:07:31+02:00             360                    M        138   

  category                   club  
0      MH1            La Sportiva  
1      MH1      TÊTE D'OR RUNNER

In [33]:
with open(f'Data/JSON/SwissPeaks360_{year}.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

participants = json_data['pageProps']['resultsResult']['hydra:member']

extracted_list = []

for p in participants:
    user = p['user']
    
    # Extract intermediate results with formatted values
    intermediate_info = {}
    for result in p.get('intermediateResults', []):
        label = result.get('label')
        value = result.get('value')
        
        # Convert numeric time values (in seconds) to hh:mm:ss
#         if isinstance(value, (int, float)):
#             value = str(timedelta(seconds=value))
        
        intermediate_info[label] = value

    # Main participant info
    extracted_info = {
        'firstName': user.get('firstName'),
        'lastName': user.get('lastName'),
        'Slug': user.get('slug'),
        'Nationality': user.get('nationality'),
        'Gender': user.get('gender'),
        'DOB': user.get('birthDate'),
        'Race': user.get('distanceCarried'),
        'participation_gender': p.get('gender'),
        'Bib': p.get('raceNumber'),
        'Age Category': p.get('category'),
        'Club': p.get('club'),
        'Status': p.get('status')
    }

    # Merge main info with intermediate results
    extracted_info.update(intermediate_info)
    extracted_list.append(extracted_info)

# Convert to DataFrame
df = pd.DataFrame(extracted_list)
df['Year'] = year
df['Race'] = 'SP' + df['Race'] 

df['Name'] = df['firstName'] + ' ' + df['lastName'] 

df['PK'] = df['Race'] +'_' + df['Year'] +'_' + df['Bib']    

# Preview the result
print( df[['PK','Bib','Name', 'Nationality', 'Gender','Age Categoty', 'DOB' 'Club', 'Status', 'Slug']].head())



  firstName     lastName                  slug nationality gender  \
0       Kim     COLLISON          collison-kim         GBR      M   
1  Matthieu  BOURGUIGNON  bourguignon-matthieu         FRA      M   
2     Jorge        GALVE           galve-jorge         ESP      M   
3    Adrien      BARBIER        barbier-adrien         FRA      M   
4     Iñigo      ORRADRE         orradre-inigo         ESP      M   

                   birthDate distanceCarried participation_gender raceNumber  \
0  1980-05-07T14:06:37+02:00             360                    M          2   
1  1979-03-26T14:07:05+01:00             360                    M         79   
2  1985-03-13T14:06:38+01:00             360                    M          5   
3  1989-02-01T09:51:16+01:00             360                    M         86   
4  1978-08-09T14:07:31+02:00             360                    M        138   

  category  ... Rang 58Tour de Don Temps 58Tour de Don  \
0      MH1  ...                  0            

In [34]:
# Preview the result
print(df[(df['firstName'] == 'Kim') & (df['lastName'] == 'COLLISON')].transpose())

                                0
firstName                     Kim
lastName                 COLLISON
slug                 collison-kim
nationality                   GBR
gender                          M
...                           ...
Temps 60Taney                   0
Rang 61Grand Pré                0
Temps 61Grand Pré               0
Rang 62Débarcadère              0
Temps 62Débarcadère             0

[105 rows x 1 columns]


In [ ]:
# End the timer
end_time = time.time()

# Calculate and print execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.6f} seconds")